In [1]:
import pandas as pd
import spacy
import re
import seaborn as sns
sns.set()
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from IPython.display import IFrame
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from scipy.spatial import distance

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-16 17:29:07.064348: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 17:29:07.969274: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-16 17:29:07.969317: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
202

In [2]:
vectorizer = TfidfVectorizer(input = 'content', strip_accents = 'ascii', stop_words = 'english')

In [3]:
data = pd.read_csv('Winddata.csv', index_col = 0)

data = data.drop_duplicates(subset = ['text']).reset_index(drop = True)

data =data.reset_index(drop=True)


In [4]:
text = [i for i in data['text']]

In [5]:
vectors = vectorizer.fit_transform(text)

In [6]:
vectors = vectors.todense().tolist()

In [7]:
df = pd.DataFrame(vectors,columns=vectorizer.get_feature_names_out())

In [8]:
pca_1 = PCA(n_components = 3)
comps_1 = pca_1.fit_transform(df)
pc_df_1 = pd.DataFrame(data = comps_1, columns = ['PC'+str(i) for i in range(1, comps_1.shape[1]+1)])

In [9]:
df.dtypes #make sure its float 64

000            float64
01             float64
02             float64
03             float64
039            float64
                ...   
zones          float64
zoning         float64
zooms          float64
zzc5cda7zxm    float64
zzzt           float64
Length: 5417, dtype: object

In [10]:
clustering = AgglomerativeClustering(n_clusters = 5, linkage = 'ward').fit(df)

In [11]:
kmeans = KMeans(n_clusters=5, random_state=0, n_init=10).fit(df)

In [12]:
df_all = pd.concat([df, pc_df_1], axis = 1)
df_all['clusters_ag'] = [str(i) for i in clustering.labels_]
df_all['clusters_knn'] = [str(i) for i in kmeans.labels_]
df_all['data_text'] = data['text']


In [13]:
fig = px.scatter_3d(df_all, x='PC1', y='PC2', z='PC3', color='clusters_ag', hover_data = ['data_text'])

fig.update_traces(marker=dict(size = 5, line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=6988881d-1932-460c-b3ad-5b8ae39e2682' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>